In [1]:
import random
import networkx as nx
import itertools

In [2]:
def load_graph_from_file(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [3]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [4]:
def fitness(resolving_set, G):
    unresolved_pairs = 0
    node_list = list(G.nodes)  # Pravi čvorovi iz grafa

    resolving_set = {node_list[i] for i in resolving_set if i < len(node_list)}  # Sigurno mapiranje

    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(nx.shortest_path_length(G, u, j) - nx.shortest_path_length(G, v, j)) > 0 for j in resolving_set):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova


In [5]:
def initialize_particles(num_particles, n):
    particles = []
    velocities = []
    
    for _ in range(num_particles):
        particle = [random.randint(0, 1) for _ in range(n)]  # Binarni niz
        velocity = [random.uniform(-1, 1) for _ in range(n)]  # Brzina svake komponente
        particles.append(particle)
        velocities.append(velocity)
    
    return particles, velocities

In [6]:
def update_particle(particle, velocity, pBest, gBest, w, c1, c2):
    new_velocity = []
    new_particle = []

    for i in range(len(particle)):
        r1 = random.random()
        r2 = random.random()

        # Ažuriranje brzine
        v_new = w * velocity[i] + c1 * r1 * (pBest[i] - particle[i]) + c2 * r2 * (gBest[i] - particle[i])
        new_velocity.append(v_new)

        # Sigmoidna aktivacija za binarni PSO
        prob = 1 / (1 + pow(2.718, -v_new))  # Sigmoidna funkcija
        new_particle.append(1 if random.random() < prob else 0)  # Konvertovanje u 0 ili 1
    
    return new_particle, new_velocity

In [7]:
def particle_swarm_optimization(G, num_particles, num_iterations, w, c1, c2):
    node_list = list(G.nodes)  # Pravi čvorovi iz grafa

    particles, velocities = initialize_particles(num_particles, len(node_list))
    pBest = particles[:]  # Najbolje pojedinačne pozicije
    pBest_fitness = [fitness({i for i, bit in enumerate(p) if bit == 1 and i < len(node_list)}, G) for p in particles]

    # Pronalazak početnog gBest
    gBest = particles[pBest_fitness.index(min(pBest_fitness))]
    gBest_fitness = min(pBest_fitness)

    for iteration in range(num_iterations):
        for i in range(num_particles):
            # Ažuriranje čestica
            particles[i], velocities[i] = update_particle(particles[i], velocities[i], pBest[i], gBest, w, c1, c2)
            current_fitness = fitness({i for i, bit in enumerate(particles[i]) if bit == 1 and i < len(node_list)}, G)

            # Ažuriranje pBest
            if current_fitness < pBest_fitness[i]:
                pBest[i] = particles[i]
                pBest_fitness[i] = current_fitness

        # Ažuriranje gBest
        best_particle_index = pBest_fitness.index(min(pBest_fitness))
        if pBest_fitness[best_particle_index] < gBest_fitness:
            gBest = pBest[best_particle_index]
            gBest_fitness = pBest_fitness[best_particle_index]

        print(f"Iteracija {iteration+1}, Najbolji fitnes: {gBest_fitness}")

        # Ako nema nerešenih parova, završavamo pre roka
        if gBest_fitness < 100:
            break

    # Prikaz rezultata
    optimal_set = {node_list[i] for i, bit in enumerate(gBest) if bit == 1 and i < len(node_list)}
    print("\nOptimalni rešavajući skup:", optimal_set)
    return optimal_set

In [8]:
# G = nx.petersen_graph()

# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

G = load_graph_from_file('grafovi\gcol1.txt')

n = len(G.nodes)

# Parametri PSO algoritma
num_particles = 30  # Broj čestica
num_iterations = 100  # Broj iteracija
w = 0.7  # Inercijalni faktor
c1 = 1.5  # Koeficijent ubrzanja ka pBest
c2 = 1.5  # Koeficijent ubrzanja ka gBest

rez = particle_swarm_optimization(G, num_particles, num_iterations, w, c1, c2)
print(is_resolving_set(rez,G))

<>:6: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\g'
C:\Users\pc\AppData\Local\Temp\ipykernel_8628\4005229057.py:6: SyntaxWarning: invalid escape sequence '\g'
  G = load_graph_from_file('grafovi\gcol1.txt')


Iteracija 1, Najbolji fitnes: 39

Optimalni rešavajući skup: {2, 3, 6, 11, 12, 17, 23, 26, 27, 30, 34, 36, 38, 41, 42, 44, 46, 65, 66, 67, 69, 70, 71, 72, 73, 74, 80, 84, 85, 86, 87, 88, 90, 91, 92, 97, 98, 99, 100}
True
